In [2]:
import os 
from glob import glob

In [28]:
import os
from concurrent.futures import ProcessPoolExecutor
from functools import partial

failed_files = []

def gdal_reproject(input_file, output_file):
    """Reproject a single file to EPSG:4326 using gdalwarp."""
    exit_code = os.system(f"gdalwarp -t_srs EPSG:4326 {input_file} {output_file}")
    return exit_code == 0

def process_file(fi, dtm_path):
    """Generate output file path and call gdal_reproject."""
    fo = f"{dtm_path}/{os.path.basename(fi)}"
    print(f"Reprojecting: {fi} -> {fo}")
    success = gdal_reproject(fi, fo)
    if not success:
        failed_files.append(fi)

def reproject_in_parallel(dtm_files, dtm_path, max_workers=None, limit=None):
    """Reproject multiple files in parallel using ProcessPoolExecutor.
    
    Args:
        dtm_files (list): List of input file paths.
        dtm_path (str): Output directory.
        max_workers (int): Number of parallel workers. Default is all available.
        limit (int): Limit the number of files to process. Default is None (no limit).
    """
    if limit:
        dtm_files = dtm_files[:limit]  # Limit the number of files if specified

    # Create a partial function with the output path
    process_with_path = partial(process_file, dtm_path=dtm_path)

    # Run the reproject function in parallel
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        executor.map(process_with_path, dtm_files)

    # Send desktop notification after completion
    os.system('notify-send "Reprojection Complete" "All files have been reprojected." -t 120000')

    # Log summary
    total_files = len(dtm_files)
    failed_count = len(failed_files)
    success_count = total_files - failed_count

    print("\nSummary:")
    print(f"Total files: {total_files}")
    print(f"Successfully reprojected: {success_count}")
    print(f"Failed to reproject: {failed_count}")

    # if failed_files:
    #     print("\nThe following files failed to reproject:")
    #     for file in failed_files:
    #         print(file)
    return failed_files

def list2txt(files, txtfile):
    with open(txtfile, "w") as file:
        for f in files:
            file.write(f"{f}\n")

In [26]:
odpath = "/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326"

In [27]:
dtm_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/*/*DTM10_mean.tif")
dsm_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/*/*DSM_10_max.tif")
len(dtm_files), len(dsm_files)

(910, 910)

In [30]:
dsm_txt = f"{odpath}/dsm_files.txt"
dtm_txt = f"{odpath}/dtm_files.txt"

dsm_gpkg = f"{odpath}/dsm_files.gpkg"
dtm_gpkg = f"{odpath}/dtm_files.gpkg"

list2txt(dsm_files, dsm_txt)
list2txt(dtm_files, dtm_txt)

In [13]:
dtm_path = f"{odpath}/DTM"
dsm_path = f"{odpath}/DSM"
os.makedirs(dtm_path, exist_ok=True)
os.makedirs(dsm_path, exist_ok=True)

In [14]:
#ffilesa =  reproject_in_parallel(dtm_files, dtm_path, max_workers=30, limit=None)

In [15]:
#ffilesb = reproject_in_parallel(dsm_files, dsm_path, max_workers=30, limit=None)

In [ ]:
import subprocess

def create_gpkg_index(output_gpkg, optfile, t_srs="EPSG:4326"):
    """
    Creates a GPKG index of raster files using gdalindex.

    Parameters:
        output_gpkg (str): Path to the output GPKG file.
        optfile (str): Path to the text file containing list of raster files.
        t_srs (str): Target spatial reference system (default: EPSG:4326).

    Returns:
        None
    """
    try:
        command = [
            "gdaltindex",
            "-f", "GPKG",
            "-write_absolute_path",
            "-t_srs", t_srs,
            "--optfile", optfile,
            output_gpkg
        ]
        subprocess.check_call(command)
        print(f"Index created successfully: {output_gpkg}")
    except subprocess.CalledProcessError as e:
        print(f"Error during gdalindex execution: {e}")


            
# Example usage
#create_gpkg_index("output_index.gpkg", "my_list.txt")


In [17]:
dtm_path = f"{odpath}/DTM"
dsm_path = f"{odpath}/DSM"

dsm_filesx = glob(f"{dsm_path}/*.tif"); print(len(dsm_filesx))  
dtm_filesx = glob(f"{dtm_path}/*.tif"); print(len(dtm_filesx))

899
899


In [ ]:



list2txt(dsm_files, dsm_txt)
list2txt(dtm_files, dtm_txt)
dtm_txt, dsm_txt

('/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/dtm_files.txt',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/dsm_files.txt')

In [24]:






create_gpkg_index(f"{odpath}/DTM.gpkg", dtm_txt)
create_gpkg_index(f"{odpath}/DSM.gpkg", dsm_txt)

Warning 1: The filename extension should be 'gpkg' instead of 'tif' to conform to the GPKG specification.
ERROR 4: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DTM.gpkg: No such file or directory
Warning 1: Unable to open /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DTM.gpkg, skipping.


Index created successfully: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DTM.gpkg


Warning 1: The filename extension should be 'gpkg' instead of 'tif' to conform to the GPKG specification.
ERROR 4: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM.gpkg: No such file or directory
Warning 1: Unable to open /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM.gpkg, skipping.


Index created successfully: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM.gpkg


In [48]:
glob(f"{dsm_path}/*.tif")

['/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0430_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0001_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0002_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0003_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0004_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0005_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0006_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0007_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0008_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0009_returnDSM_10_max.tif',
 '/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF4326/DSM/NP_T-0010_